In [1]:
import os
import re
import rasterio

# Path to the folder containing the monthly TIFF files
monthly_folder = r'/efs/Lerma/new_data_set/NPP'

# Output folder for the new yearly data
output_folder = r'/efs/Lerma/new_data_set/yearly/SSEBOP_ET'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all TIFF files in the monthly folder
monthly_tif_files = [file for file in os.listdir(monthly_folder) if file.endswith('.tif')]

# Create a dictionary to store monthly data for each year
yearly_data = {}

# Loop through the monthly TIFF files
for monthly_file in monthly_tif_files:
    monthly_file_path = os.path.join(monthly_folder, monthly_file)
    
    # Extract the year from the file name using a regular expression
    year_match = re.search(r'\d{4}', monthly_file)
    if year_match:
        year = year_match.group(0)
        
        # Open the monthly data file
        with rasterio.open(monthly_file_path) as monthly_src:
            # Read the data from the monthly file
            monthly_data = monthly_src.read(1)
            
            # If the year is not in the yearly_data dictionary, create an entry
            if year not in yearly_data:
                yearly_data[year] = monthly_data
                print(year)
            else:
                # Add the monthly data to the existing yearly data
                yearly_data[year] += monthly_data

# Loop through the yearly data dictionary and save the results as new TIFF files
for year, data in yearly_data.items():
    output_file = f'ssebop_{year}.01.01.tif'
    output_file_path = os.path.join(output_folder, output_file)
    
#     data = data/12
    
    # Use the metadata from one of the monthly files (assuming they have the same metadata)
    metadata = None
    for monthly_file in monthly_tif_files:
        if year in monthly_file:
            with rasterio.open(os.path.join(monthly_folder, monthly_file)) as src:
                metadata = src.profile
                break
    
    if metadata is not None:
        with rasterio.open(output_file_path, 'w', **metadata) as dst:
            dst.write(data, 1)

print("Yearly data has been created in the output folder.")


2018
2020
2014
2016
2022
2017
2012
2013
2023
2021
2015
2019
Yearly data has been created in the output folder.
